In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from datetime import datetime
import seaborn as sns
%matplotlib inline

In [2]:
import os
os.chdir("C:\\Users\\vsm397\\Desktop\\Vir's Personal documents\\Python Class- Summer 2018\\Python Project")

In [3]:
weather = pd.read_csv('austin_weather.csv').drop(['DewPointHighF','DewPointLowF','HumidityHighPercent','HumidityLowPercent',
                                               'SeaLevelPressureHighInches','SeaLevelPressureLowInches','VisibilityHighMiles',
                                               'VisibilityLowMiles','WindHighMPH','WindGustMPH','HumidityAvgPercent'],1)

trips=pd.read_csv('austin_bikeshare_trips.csv')
station=pd.read_csv('austin_bikeshare_stations_w_zipcode.csv')

In [4]:
weather['Rain']=np.where(weather.Events.str.contains('Rain'), 1, 0)
weather['Fog']=np.where(weather.Events.str.contains('Fog'), 1, 0)
weather['Thunderstorm']=np.where(weather.Events.str.contains('Thunderstorm'), 1, 0)
weather['Snow']=np.where(weather.Events.str.contains('Snow'), 1, 0)
weather['None']=np.where(weather.Events.isnull, 1, 0)
weather.Date=pd.to_datetime(weather.Date)
weather=weather.set_index('Date').drop('Events',1)

In [5]:
ind = pd.DatetimeIndex(trips.start_time)
trips['Date'] = ind.date.astype('datetime64')
trips['hour'] = ind.hour
by_date = trips.pivot_table('trip_id', aggfunc='count',index='Date',columns='subscriber_type', )
by_date = trips.groupby(['Date', 'subscriber_type'])['trip_id'].count()
by_date.name = 'count'
by_date = by_date.reset_index('subscriber_type')
trips_2=trips[trips.year.isin([2014,2015,2016])]
trip_weather=trips_2.merge(weather,left_on='Date',right_on='Date',
                         suffixes=['_trips','_weather']).drop(['SeaLevelPressureAvgInches'],1)

In [6]:
trip_weather["start_time"] = trip_weather["start_time"].map(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trip_weather["hours"] = trip_weather["start_time"].map(lambda x : x.hour)


In [7]:
val=''
def time_of_day(x):
    global val
    if (x>=6)&(x<10):
        val='morning rush hours'
    elif (x>=15)&(x<19):
        val='afternoon rush hours'
    elif (x>=10)&(x<15):
        val='noon'
    elif (x>=19)|(x<6):
        val='night'
    return val
trip_weather['Time of the day'] = trip_weather['hours'].apply(lambda a: time_of_day(a))



In [8]:
trip_zipcode=trip_weather.merge(station,left_on='start_station_id',right_on='station_id')

In [9]:
trip_zipcode.loc[trip_zipcode['zipcode'] != 78705, 'University_vicinity'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'University_vicinity'] = 1
trip_zipcode.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,Time of the day,Unnamed: 0,latitude,location,longitude,name,station_id,status,zipcode,University_vicinity
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,night,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0


In [10]:
holidays = ['2014-01-01', '2015-01-01', '2016-01-01','2014-01-18', '2014-01-19','2014-01-20','2015-01-17', 
            '2015-01-18','2015-01-19','2016-01-16', '2016-01-17','2016-01-18','2014-02-15', '2014-02-16', '2014-02-17',
            '2015-02-14', '2015-02-15', '2015-02-16','2016-02-13', '2016-02-14', '2016-02-15',
            '2014-05-24', '2014-05-25', '2014-05-26','2015-05-23', '2015-05-24', '2015-05-25','2016-05-28', '2016-05-29', 
            '2016-05-30','2014-07-04', '2014-07-05', '2014-07-06','2015-07-03', '2015-07-04', '2015-07-05',
            '2016-07-02', '2016-07-03', '2016-07-04','2014-08-30', '2014-08-31', '2014-09-01','2015-09-05', '2015-09-06',
            '2015-09-07','2016-09-03', '2016-09-04', '2016-09-05','2014-10-11', '2014-10-12', '2014-10-13',
            '2015-10-10', '2015-10-11', '2015-10-12','2016-10-08', '2016-10-09', '2016-10-10','2014-11-11', '2015-11-11',
            '2016-11-11','2014-11-27', '2014-11-28', '2014-11-29', '2014-11-30','2015-11-26', '2015-11-27', '2015-11-28',
            '2015-11-29','2016-11-24', '2016-11-25', '2016-11-26', '2016-11-27','2014-12-25', '2014-12-26', '2014-12-27',
            '2014-12-28','2015-12-25', '2015-12-26', '2015-12-27', '2015-12-28','2016-12-24', '2016-12-25', '2016-12-26',
            '2016-12-27','2014-03-07', '2014-03-16','2015-03-13', '2015-03-22', '2016-12-27','2014-03-07', '2014-03-16',
            '2015-03-13', '2015-03-22','2016-03-11', '2016-03-20','2014-10-03', '2014-10-05','2014-10-10', '2014-10-12',
            '2015-10-02', '2015-10-04','2015-10-09', '2015-10-11','2016-09-30', '2016-10-02','2016-10-07', '2016-10-09']
trip_zipcode.loc[-trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 0
trip_zipcode.loc[trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 1
trip_zipcode.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,Unnamed: 0,latitude,location,longitude,name,station_id,status,zipcode,University_vicinity,Holiday_flag
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,7,30.26461,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0


In [11]:
trip_zipcode['weekday'] = trip_zipcode['Date'].apply(lambda x: x.weekday())
trip_zipcode.loc[trip_zipcode['weekday'].isin([1,2,3,4,5]), 'Weekday_flag'] = 1
trip_zipcode.loc[trip_zipcode['weekday'].isin([6,7]), 'Weekday_flag'] = 0
trip_zipcode

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,location,longitude,name,station_id,status,zipcode,University_vicinity,Holiday_flag,weekday,Weekday_flag
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
5,894.0,17:12:00,7,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2536.0,Waller & 6th St.,2015-03-19 17:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
6,974.0,12:12:00,7,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 12:12:00,Explorer,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
7,284.0,19:12:00,11,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
8,54.0,23:12:00,59,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 23:12:00,Local30,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0
9,881.0,18:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 18:12:00,Walk Up,...,(30.26461 -97.73049),-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0


In [12]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Tourist_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Tourist_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Tourist_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Tourist_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Tourist_Location'] = 0
trip_zipcode

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,longitude,name,station_id,status,zipcode,University_vicinity,Holiday_flag,weekday,Weekday_flag,Tourist_Location
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
5,894.0,17:12:00,7,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2536.0,Waller & 6th St.,2015-03-19 17:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
6,974.0,12:12:00,7,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 12:12:00,Explorer,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
7,284.0,19:12:00,11,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
8,54.0,23:12:00,59,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 23:12:00,Local30,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0
9,881.0,18:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 18:12:00,Walk Up,...,-97.73049,Waller & 6th St.,2536,closed,78702,0.0,0.0,3,1.0,1.0


In [30]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Residential_Location'] = 1
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Residential_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Residential_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Residential_Location'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Residential_Location'] = 0

In [32]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Pop_Age_15-24'] = 2862
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Pop_Age_15-24'] = 2881
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Pop_Age_15-24'] = 1650
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Pop_Age_15-24'] = 6345
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Pop_Age_15-24'] = 24662
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Pop_Age_15-24'] = 7119
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Pop_Age_15-24'] = 2529
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Pop_Age_15-24'] = 3604
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Pop_Age_15-24'] = 2455

In [33]:
trip_zipcode.loc[trip_zipcode['zipcode'] == 78701, 'Walk_to_Work'] = 745
trip_zipcode.loc[trip_zipcode['zipcode'] == 78702, 'Walk_to_Work'] = 376
trip_zipcode.loc[trip_zipcode['zipcode'] == 78703, 'Walk_to_Work'] = 449
trip_zipcode.loc[trip_zipcode['zipcode'] == 78704, 'Walk_to_Work'] = 656
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'Walk_to_Work'] = 2801
trip_zipcode.loc[trip_zipcode['zipcode'] == 78745, 'Walk_to_Work'] = 628
trip_zipcode.loc[trip_zipcode['zipcode'] == 78746, 'Walk_to_Work'] = 218
trip_zipcode.loc[trip_zipcode['zipcode'] == 78751, 'Walk_to_Work'] = 158
trip_zipcode.loc[trip_zipcode['zipcode'] == 78752, 'Walk_to_Work'] = 222

In [35]:
#trip_zipcode = trip_zipcode.drop(columns=["None", "Unnamed: 0"])

In [37]:
trip_zipcode.to_csv("Analytical DF Bike Share.csv")

In [41]:
Analytical_DF_Bike_Share_1 = trip_zipcode.loc[1:84313]
Analytical_DF_Bike_Share_2 = trip_zipcode.loc[84314:168626]
Analytical_DF_Bike_Share_3 = trip_zipcode.loc[168627:252939]
Analytical_DF_Bike_Share_4 = trip_zipcode.loc[252940:337252]
Analytical_DF_Bike_Share_5 = trip_zipcode.loc[337253:421565]
Analytical_DF_Bike_Share_6 = trip_zipcode.loc[421566:505878]

In [28]:
Analytical_DF_Bike_Share_1.to_csv("ADS_1.csv")
Analytical_DF_Bike_Share_2.to_csv("ADS_2.csv")
Analytical_DF_Bike_Share_3.to_csv("ADS_3.csv")
Analytical_DF_Bike_Share_4.to_csv("ADS_4.csv")
Analytical_DF_Bike_Share_5.to_csv("ADS_5.csv")
Analytical_DF_Bike_Share_6.to_csv("ADS_6.csv")

In [29]:
#trip_zipcode.groupby("zipcode")

In [44]:
clean = pd.read_csv("Analytical DF Bike Share CLEANED.csv")
clean

,Unnamed: 0,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,...,status,zipcode,University_vicinity,Holiday_flag,weekday,Weekday_flag,Tourist_Location,Residential_Location,Pop_Age_15-24,Walk_to_Work
0,0,8,19:12:00,41,2565.0,Trinity & 6th Street,3,2536,Waller & 6th St.,3/19/2015 19:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
1,1,371,19:12:00,6,2822.0,East 6th at Robert Martinez,3,2536,Waller & 6th St.,3/19/2015 19:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
2,2,922,19:12:00,31,2495.0,4th & Congress,3,2536,Waller & 6th St.,3/19/2015 19:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
3,3,991,21:12:00,25,2544.0,East 6th & Pedernales St.,3,2536,Waller & 6th St.,3/19/2015 21:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
4,4,822,21:12:00,0,2536.0,Waller & 6th St.,3,2536,Waller & 6th St.,3/19/2015 21:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
5,5,894,17:12:00,7,2498.0,Convention Center / 4th St. @ MetroRail,3,2536,Waller & 6th St.,3/19/2015 17:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
6,6,974,12:12:00,7,2565.0,Trinity & 6th Street,3,2536,Waller & 6th St.,3/19/2015 12:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
7,7,284,19:12:00,11,2498.0,Convention Center / 4th St. @ MetroRail,3,2536,Waller & 6th St.,3/19/2015 19:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
8,8,54,23:12:00,59,2565.0,Trinity & 6th Street,3,2536,Waller & 6th St.,3/19/2015 23:12,...,closed,78702,0,0,3,1.0,1,1,2881,376
9,9,881,18:12:00,6,2822.0,East 6th at Robert Martinez,3,2536,Waller & 6th St.,3/19/2015 18:12,...,closed,78702,0,0,3,1.0,1,1,2881,376


In [47]:
Clean_1 = clean.loc[1:72055]
Clean_2 = clean.loc[72056:144110]
Clean_3 = clean.loc[144111:216165]
Clean_4 = clean.loc[216166:288220]
Clean_5 = clean.loc[288221:360275]
Clean_6 = clean.loc[360276:432330]
Clean_7 = clean.loc[432331:504387]

In [48]:
Clean_1.to_csv("Bike_Share_DF_CLEANED_1.csv")
Clean_2.to_csv("Bike_Share_DF_CLEANED_2.csv")
Clean_3.to_csv("Bike_Share_DF_CLEANED_3.csv")
Clean_4.to_csv("Bike_Share_DF_CLEANED_4.csv")
Clean_5.to_csv("Bike_Share_DF_CLEANED_5.csv")
Clean_6.to_csv("Bike_Share_DF_CLEANED_6.csv")
Clean_7.to_csv("Bike_Share_DF_CLEANED_7.csv")